In [ ]:
"""Use of shanghai_platform
one simulation, several draws"""


#see numpy and random

In [1]:
# 0.1 Modules

from CircleNet.system import Simulation
from CircleNet.shape import circle,Timer
from CircleNet.animation import format_time,Displayer
from CircleNet.shanghai_platform import MatchingPlatform, Driver, Passenger
from CircleNet.shanghai_platform_extension import Positions_drawing, plot_routes, plot_relative_routes,extract_result_data
import numpy as np

In [2]:
# 0.2 Help

#help(MatchingPlatform)
#print(Driver.attributes), print(Passenger.attributes)

In [3]:
# 1.1 Parameters

speed= 25 #km/h
R= 25 #km
end= 3 #h

N_driver= 600
N_passenger= 600


#in second :
#Drivers:
first_watching_before_first_departure= 5 * 60
window_size_of_departure= 15 * 60
time_elasticity= 5 * 60
fuel_cost= 0.5
watching_repetition_average= 60 # -> random
watching_repetition_variance= 10 # poisson ?
time_perception_average= 1 / 300 # = 24 * 50/100 / 3600 #50% of average income, in second   -> random
time_perception_variance= 1 / 3000 #10% percent

#Passenger
publishing_advance= 20 * 60

In [4]:
# 1.2 Simulation type

#transformation in m, s
speed/=3.6
R*=1000
end*=3600


def benefits(origin,destination,network):
    """Shanghai pricing"""
    distance=network.travel_distance(origin,destination)
    if distance < 3000: #3n first km fixprice
        return 11#in RMB
    if distance < 20000:#until 20km at 1.5RMB / km
        return 6.5 + distance * 1.5 # 11 + (distance - 3) * 1.5
    return 8.5 + distance # 28.5 + (distance - 20) *1   # 28.5 = 3 + (20 - 3) * 1.5

def get_log_normal(average,variance):
    sigma=np.log(1+(variance/average)**2)
    mu=np.log(average)-sigma**2/2
    sigma=np.sqrt(sigma)
    def out():
        return np.random.lognormal(mu, sigma)
    return out
watching_repetition=get_log_normal(watching_repetition_average,watching_repetition_variance)
time_perception=get_log_normal(time_perception_average,time_perception_variance)


#agents generator
def SimpleDriver(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    w=(t+first_watching_before_first_departure,t+first_watching_before_first_departure+window_size_of_departure)
    A=w[1]+simulation.network.travel_time(O,D)+time_elasticity
    return Driver(first_watching_time=t,
                  repetition_time=watching_repetition(),
                  departure_window=w,
                  position=O,destination=D,
                  last_arrival_time=A,
                  fuel_cost=fuel_cost,
                  time_perception=time_perception())
def SimplePassenger(simulation):
    t=simulation.timer.random_time()
    O=simulation.network.position_generator()
    D=simulation.network.position_generator()
    return Passenger(publishing_time=t,
                     last_departure_time=t+publishing_advance,
                     position=O,destination=D)


In [5]:
#1.3 Simulation construction

N=circle(R,speed)
T=Timer(end)
simu=Simulation(N,T)
d=None

simu.matchingAlgo=MatchingPlatform(benefits,simu)


for i in range(N_driver):
    simu.add(SimpleDriver(simu))
for i in range(N_passenger):
    simu.add(SimplePassenger(simu))
    



In [6]:
#2.1 Time control
# OPTIONAL

def f(u):
    return format_time(float(u))
d=Displayer(text='0',function_when_display=f)
d.start()

In [7]:
#2.2 Execution



#simu.set_action(print,)
if d:
    simu.set_action(lambda x: d.change(x.time))

    

simu()
print(simu.execution_time)

0.8693020343780518


In [8]:
#2.3 Close time controler

if d:
    d.quit()

In [9]:
#3.1 Observations

results=extract_result_data(simu)
print(results)

#histogram for relative standard deviation

#drivers characteristic on his behavior
#same for passenger
#relation between general scenario and matching efficiency

{'average_vks': 13721.982550474728, 'average_waiting_time': 100.89455117515685, 'passenger_efficiency': 0.23833333333333334, 'nb_driver': 600, 'total_vks': 1962243.504717886, 'id_sim': '2016111131239', 'driver_efficiency': 0.23833333333333334, 'nb_match': 143, 'nb_passenger': 600, 'execution_time': 0.8693020343780518}


In [11]:
# 3.2 Animation

dr=Positions_drawing(simu)
dr.set_options(margin=20,real_size=(10,10),time_interval=30,time_coef=100,repetition=True,hide_axes=True)
#%matplotlib notebook
dr()

In [14]:
# 3.3 Route representation

# a. All trajectories
plot_routes(simu)



In [15]:
# b. Relative route
plot_relative_routes(simu)

In [13]:
for p in simu:
    print(p)

Agent 1 of type Driver.
begin at 8957.752253523016 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9023.703901466586 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9089.655549410156 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9155.607197353725 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9221.558845297295 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9287.510493240865 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9353.462141184435 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9419.413789128004 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9485.365437071574 to ('watching', {'position': (9714.679753733028, -4187.648470756118)}) 
begin at 9551.317085015144 to ('watching', {'position': (9714.679753733028, -4187.648